In [2]:
import collections
from datasets import load_dataset

ds_math = load_dataset("open-r1/OpenThoughts-114k-math")

# 1. Source distribution — what kind of math problems?
sources = collections.Counter(ex['source'] for ex in ds_math['train'])
print("=== SOURCES ===")
for source, count in sources.most_common(20):
    print(f"  {source}: {count}")

# 2. Token count distribution (already computed for us!)
token_counts = [ex['generated_token_count'] for ex in ds_math['train']]
token_counts.sort()
n = len(token_counts)

print(f"\n=== TOKEN COUNT DISTRIBUTION ===")
print(f"Total examples: {n}")
print(f"Min: {token_counts[0]}")
print(f"10th percentile: {token_counts[n//10]}")
print(f"25th percentile: {token_counts[n//4]}")
print(f"50th percentile: {token_counts[n//2]}")
print(f"75th percentile: {token_counts[3*n//4]}")
print(f"90th percentile: {token_counts[9*n//10]}")
print(f"Max: {token_counts[-1]}")

# 3. Our difficulty splits using ACTUAL token counts
p25 = token_counts[n//4]
p75 = token_counts[3*n//4]
easy = [ex for ex in ds_math['train'] if ex['generated_token_count'] < p25]
medium = [ex for ex in ds_math['train'] if p25 <= ex['generated_token_count'] < p75]
hard = [ex for ex in ds_math['train'] if ex['generated_token_count'] >= p75]

print(f"\n=== DIFFICULTY SPLITS ===")
print(f"Easy  (< {p25} tokens): {len(easy)}")
print(f"Medium ({p25}-{p75} tokens): {len(medium)}")
print(f"Hard  (> {p75} tokens): {len(hard)}")

# 4. Look at one easy example
print(f"\n=== EASY EXAMPLE ===")
easy_sorted = sorted(easy, key=lambda x: x['generated_token_count'])
ex = easy_sorted[0]
print(f"Source: {ex['source']}")
print(f"Tokens: {ex['generated_token_count']}")
print(f"Problem: {ex['problem'][:200]}")
thinking = ex['conversations'][1]['value']
if '<|end_of_thought|>' in thinking:
    think_part = thinking.split('<|end_of_thought|>')[0].replace('<|begin_of_thought|>', '').strip()
    print(f"\nThinking ({len(think_part.split())} words): {think_part[:300]}...")

=== SOURCES ===
  olympiads: 70959
  aops_forum: 9096
  math: 6580
  amc_aime: 2485

=== TOKEN COUNT DISTRIBUTION ===
Total examples: 89120
Min: 285
10th percentile: 1770
25th percentile: 2820
50th percentile: 4957
75th percentile: 8648
90th percentile: 13304
Max: 34179

=== DIFFICULTY SPLITS ===
Easy  (< 2820 tokens): 22272
Medium (2820-8648 tokens): 44567
Hard  (> 8648 tokens): 22281

=== EASY EXAMPLE ===
Source: olympiads
Tokens: 285
Problem: Kiyana gives half of her 24 grapes to a friend. How many grapes does she give away?

Thinking (137 words): Okay, let's see. Kiyana has 24 grapes, and she gives half of them to her friend. The question is asking how many grapes she gives away. Hmm. So first, I need to figure out what half of 24 is. Half means dividing by 2, right? So if you split 24 into two equal parts, each part would be half. Let me do...


In [3]:
# Sort by token count
all_examples = list(ds_math['train'])
all_examples.sort(key=lambda x: x['generated_token_count'])

# Show the easiest example
print("=== EASIEST EXAMPLE ===")
ex = all_examples[0]
print(f"Tokens: {ex['generated_token_count']} | Source: {ex['source']}")
print(f"Problem: {ex['problem'][:300]}")
thinking = ex['conversations'][1]['value']
think_part = thinking.split('<|end_of_thought|>')[0].replace('<|begin_of_thought|>', '').strip()
answer_part = thinking.split('<|end_of_thought|>')[1] if '<|end_of_thought|>' in thinking else ""
print(f"\nFull thinking ({len(think_part.split())} words):")
print(think_part[:500])
print(f"\n...answer: {answer_part[:200]}")

# Show a median example
print(f"\n\n{'='*60}")
print("=== MEDIAN EXAMPLE ===")
ex = all_examples[len(all_examples)//2]
print(f"Tokens: {ex['generated_token_count']} | Source: {ex['source']}")
print(f"Problem: {ex['problem'][:300]}")
thinking = ex['conversations'][1]['value']
think_part = thinking.split('<|end_of_thought|>')[0].replace('<|begin_of_thought|>', '').strip()
print(f"\nThinking length: {len(think_part.split())} words")
print(f"First 300 chars: {think_part[:300]}")

# Show a hard example
print(f"\n\n{'='*60}")
print("=== HARDEST EXAMPLE ===")
ex = all_examples[-1]
print(f"Tokens: {ex['generated_token_count']} | Source: {ex['source']}")
print(f"Problem: {ex['problem'][:300]}")
thinking = ex['conversations'][1]['value']
think_part = thinking.split('<|end_of_thought|>')[0].replace('<|begin_of_thought|>', '').strip()
print(f"\nThinking length: {len(think_part.split())} words")
print(f"First 300 chars: {think_part[:300]}")

=== EASIEST EXAMPLE ===
Tokens: 285 | Source: olympiads
Problem: Kiyana gives half of her 24 grapes to a friend. How many grapes does she give away?

Full thinking (137 words):
Okay, let's see. Kiyana has 24 grapes, and she gives half of them to her friend. The question is asking how many grapes she gives away. Hmm. So first, I need to figure out what half of 24 is. Half means dividing by 2, right? So if you split 24 into two equal parts, each part would be half. Let me double-check that. 24 divided by 2 equals 12. Yeah, because 12 times 2 is 24. So if she gives half away, that's 12 grapes. Wait, is there another way to look at this? Maybe using multiplication. Half is

...answer: 

<|begin_of_solution|>

Kiyana starts with **24 grapes**. Giving half to her friend involves dividing the total by 2:  
\[
\frac{24}{2} = 12 \text{ grapes}
\]  
**Final Answer:** Kiyana gives away \b


=== MEDIAN EXAMPLE ===
Tokens: 4957 | Source: olympiads
Problem: Prove that the sum of two consecutive powe

In [4]:
for i in range(2):
    ex = ds_math['train'][i]
    print(f"{'='*70}")
    print(f"ROW {i}")
    print(f"{'='*70}")
    for key in ex.keys():
        val = ex[key]
        if isinstance(val, str):
            print(f"\n[{key}] ({len(val)} chars):")
            print(val[:400])
            if len(val) > 400:
                print(f"...\n")
        elif isinstance(val, list):
            print(f"\n[{key}] (list, {len(val)} items):")
            for j, item in enumerate(val):
                print(f"  item {j}: {str(item)[:300]}")
                if len(str(item)) > 300:
                    print(f"  ...")
        else:
            print(f"\n[{key}]: {val}")
    print("\n\n")

ROW 0

[source] (9 chars):
olympiads

[problem] (196 chars):
Let \( a, b, c \) be positive real numbers. Prove that

$$
\frac{1}{a(1+b)}+\frac{1}{b(1+c)}+\frac{1}{c(1+a)} \geq \frac{3}{1+abc},
$$

and that equality occurs if and only if \( a = b = c = 1 \).

[solution] (2719 chars):
1. Consider the given inequality:

\[
\frac{1}{a(1+b)}+ \frac{1}{b(1+c)}+ \frac{1}{c(1+a)} \geq \frac{3}{1 + abc}
\]

To simplify, we add \( \frac{3}{1 + abc} \) to both sides. The new inequality becomes:

\[
\frac{1}{a(1+b)} + \frac{1}{b(1+c)} + \frac{1}{c(1+a)} + \frac{3}{1 + abc} \geq \frac{6}{1 + abc}
\]

2. Let's analyze each term with an added \( \frac{1}{1 + abc} \):

\[
\frac{1}{a(1+b)} + 
...


[messages] (list, 2 items):
  item 0: {'content': 'Let \\( a, b, c \\) be positive real numbers. Prove that\n\n$$\n\\frac{1}{a(1+b)}+\\frac{1}{b(1+c)}+\\frac{1}{c(1+a)} \\geq \\frac{3}{1+abc},\n$$\n\nand that equality occurs if and only if \\( a = b = c = 1 \\).', 'role': 'user'}
  item 1: {'content': "1. Co

In [5]:
ex = ds_math['train'][0]

# The conversations column has the thinking
convo = ex['conversations']
print("Number of turns:", len(convo))

for j, turn in enumerate(convo):
    print(f"\n{'='*50}")
    print(f"TURN {j} — role: {turn['from']}")
    print(f"Length: {len(turn['value'])} chars")
    print(f"{'='*50}")
    print(turn['value'][:1000])

Number of turns: 2

TURN 0 — role: user
Length: 240 chars
Return your final response within \boxed{}. Let \( a, b, c \) be positive real numbers. Prove that

$$
\frac{1}{a(1+b)}+\frac{1}{b(1+c)}+\frac{1}{c(1+a)} \geq \frac{3}{1+abc},
$$

and that equality occurs if and only if \( a = b = c = 1 \).

TURN 1 — role: assistant
Length: 42742 chars
<|begin_of_thought|>

Okay, so I need to prove this inequality: \(\frac{1}{a(1+b)}+\frac{1}{b(1+c)}+\frac{1}{c(1+a)} \geq \frac{3}{1+abc}\), where \(a, b, c\) are positive real numbers, and equality holds if and only if \(a = b = c = 1\). Hmm, let me try to think through this step by step.

First, I remember that inequalities often involve techniques like AM-GM, Cauchy-Schwarz, or other classical inequalities. Maybe I can start by looking at each term on the left side and see if I can manipulate them to relate to the right side.

The left side has three terms: \(\frac{1}{a(1+b)}\), \(\frac{1}{b(1+c)}\), and \(\frac{1}{c(1+a)}\). Each denominator h

In [6]:
response = ds_math['train'][0]['conversations'][1]['value']

print("Contains <|begin_of_thought|>:", '<|begin_of_thought|>' in response)
print("Contains <|end_of_thought|>:", '<|end_of_thought|>' in response)
print("Contains <|begin_of_solution|>:", '<|begin_of_solution|>' in response)
print("Contains <|end_of_solution|>:", '<|end_of_solution|>' in response)

# Split it
parts = response.split('<|end_of_thought|>')
thinking = parts[0].replace('<|begin_of_thought|>', '').strip()
answer = parts[1].strip() if len(parts) > 1 else "N/A"

print(f"\nThinking: {len(thinking)} chars")
print(f"Answer: {len(answer)} chars")

print(f"\nFirst 200 chars of thinking:\n{thinking[:200]}")
print(f"\nLast 200 chars of thinking:\n{thinking[-200:]}")
print(f"\nFull answer:\n{answer[:500]}")

Contains <|begin_of_thought|>: True
Contains <|end_of_thought|>: True
Contains <|begin_of_solution|>: True
Contains <|end_of_solution|>: True

Thinking: 41079 chars
Answer: 1619 chars

First 200 chars of thinking:
Okay, so I need to prove this inequality: \(\frac{1}{a(1+b)}+\frac{1}{b(1+c)}+\frac{1}{c(1+a)} \geq \frac{3}{1+abc}\), where \(a, b, c\) are positive real numbers, and equality holds if and only if \(

Last 200 chars of thinking:


4. **Equality Condition:**
   - Equality in AM-GM occurs when all terms are equal, leading to \(a = b = c = 1\).

Thus, the inequality is proven, and equality holds exclusively at \(a = b = c = 1\).

Full answer:
<|begin_of_solution|>

To prove the inequality 

\[
\frac{1}{a(1+b)} + \frac{1}{b(1+c)} + \frac{1}{c(1+a)} \geq \frac{3}{1+abc}
\]

for positive real numbers \(a, b, c\) with equality if and only if \(a = b = c = 1\), we proceed as follows:

### Key Steps and Proof:

1. **Substitution for \(abc = 1\):**
   - Assume \(abc = 1\). Let \(a = \

````
conversations[1]['value'] = 

    <|begin_of_thought|>          ← THINKING STARTS
    Okay, so I need to prove...
    ... (42,742 chars of reasoning)
    <|end_of_thought|>            ← THINKING ENDS

    <|begin_of_solution|>         ← FINAL ANSWER STARTS
    The clean proof...
    <|end_of_solution|>           ← FINAL ANSWER ENDS
````

In [7]:
# Find a short/easy example so it fits on screen
all_examples = sorted(ds_math['train'], key=lambda x: x['generated_token_count'])
ex = all_examples[0]  # shortest example

print(f"generated_token_count: {ex['generated_token_count']}")
print(f"source: {ex['source']}")
print(f"correct: {ex['correct']}")
print(f"\n{'='*70}")
print("problem:")
print('='*70)
print(ex['problem'])
print(f"\n{'='*70}")
print("solution:")
print('='*70)
print(ex['solution'])
print(f"\n{'='*70}")
print("conversations[0] (user):")
print('='*70)
print(ex['conversations'][0]['value'])
print(f"\n{'='*70}")
print("conversations[1] (assistant) — THIS IS THE FULL RAW RESPONSE:")
print('='*70)
print(ex['conversations'][1]['value'])

generated_token_count: 285
source: olympiads
correct: False

problem:
Kiyana gives half of her 24 grapes to a friend. How many grapes does she give away?

solution:

1. Calculate half of the total number of grapes Kiyana has.
   
   Since Kiyana has 24 grapes in total, we start with:
   \[
   \text{Total grapes} = 24
   \]

2. Determine half of 24.
   \[
   \frac{24}{2} = 12
   \]
   
3. Thus, Kiyana gives away 12 grapes to her friend.

### Conclusion:
\[
\boxed{12}
\]

So the correct answer is \( \boxed{D} \).

conversations[0] (user):
Return your final response within \boxed{}. Kiyana gives half of her 24 grapes to a friend. How many grapes does she give away?

conversations[1] (assistant) — THIS IS THE FULL RAW RESPONSE:
<|begin_of_thought|>

Okay, let's see. Kiyana has 24 grapes, and she gives half of them to her friend. The question is asking how many grapes she gives away. Hmm. So first, I need to figure out what half of 24 is. Half means dividing by 2, right? So if you split 24 